In [ ]:
import struct
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

**Objective and goal for this lab**

For this lab, we're going to see how we can transform images to a format we're used to working with, and can thus use for training classification (or regression) models.

We will be using the MNIST-dataset, which is a famous dataset of handwritten black & white digits between 0-9. Our goal will be to train a classifier to correctly classify each image as the digit it portrays.

You can read more about MNIST [here](https://en.wikipedia.org/wiki/MNIST_database).

---

**Load the data**

In [ ]:
import struct

def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        _, num, rows, cols = struct.unpack(">IIII", f.read(16))
        return np.fromfile(f, dtype=np.uint8).reshape(num, rows, cols)

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        _, num = struct.unpack(">II", f.read(8))
        return np.fromfile(f, dtype=np.uint8)

# Load training data
train_images = load_mnist_images("../data/mnist/train-images.idx3-ubyte")
train_labels = load_mnist_labels("../data/mnist/train-labels.idx1-ubyte")

# Load test data
test_images = load_mnist_images("../data/mnist/t10k-images.idx3-ubyte")
test_labels = load_mnist_labels("../data/mnist/t10k-labels.idx1-ubyte")

Let's investigate what we just loaded

In [ ]:
print('train_images shape :', train_images.shape)
print('test_images shape  :', test_images.shape, end='\n\n')

print('train_labels shape :', train_labels.shape)
print('test_labels shape  :', test_labels.shape)

So both train and test images are 3D-arrays. Let's pick the first item in train_images and investigate it

In [ ]:
print('train_images[0] :', train_images[0].shape)

So, the first (out of 60.000) items in train_images is a 28x28 array. Actually, it's an image of size 28x28 pixels.

We can visualise this easily

In [ ]:
plt.imshow(train_images[0], cmap='gray');
plt.title(f'Label: {train_labels[0]}')
plt.show()

As you might've noticed, the train_labels contain the true labels (i.e., the targets) for each handwritten image.

Let's plot some more images

In [ ]:
import random

number_of_images = 10

indices = random.sample(range(60000), number_of_images)

fig, axes = plt.subplots(1, number_of_images, figsize=(15, 3))

for idx, ax in zip(indices, axes):
    ax.imshow(train_images[idx], cmap='gray')
    ax.set_title(f'Label: {train_labels[idx]}')
    ax.axis('off')

plt.show()

In [ ]:
train_images[0]

---

Ok, so how do we transform this data into a format that we can train the models we've learnt about thus far? 

Well,  what we'll do is that we're going to use *every pixel as it's own feature*. Since each image is 28x28 pixels, we're going to have 28x28 = 784 features for each image.

We can use flatten() to transform a 2D (28,28) shaped array to a single 1D (784) array.

In [ ]:
# easy transformation

train_images[0].flatten().shape

In [ ]:
# now we do it for all the images in the train_images

train_images_flattened = [list(image.flatten()) for image in train_images]

In [ ]:
train_images_flattened_array = np.array([image.flatten() for image in train_images])

# transform that list to a dataframe so that we can see what's going on
X_train_val = pd.DataFrame(train_images_flattened_array)

X_train_val


In [ ]:
X_train_val.describe()

In [ ]:
# might aswell make the labels into a dataframe too

y_train_val = pd.DataFrame(train_labels)

y_train_val

In [ ]:
y_train_val.value_counts()

Cool, no we're on familiar ground! We have our X_train_val (features) and y_train_val (targets) in a suitable format.

Now we can try training some multiclass classification models on this data!

---

## Challenges

**Task 1**

Transform test_images and test_labels similarly as above, and save them as X_test and y_test.

This is the test set you will use at the end, to assess final performance!

In [ ]:
# now we do it for all the images in the test_images

test_images_flattened = [list(image.flatten()) for image in test_images]

test_images_flattened_array = np.array([image.flatten() for image in test_images])

# transform that list to a dataframe so that we can see what's going on
X_test = pd.DataFrame(test_images_flattened_array)

X_test

In [ ]:
y_test = pd.DataFrame(test_labels)

y_test

**Task 1.5 (bonus**)

Let's normalize our features! Because why not. It won't hurt the algorithms that don't require it - and we might aswell show how easily images can be scaled properly.

In the case for gray-scale images, we simply divide all features (pixels) by 255. Why? Because that's the maximum possible value. This will bring all grayscale values to the range [0,1].

In [ ]:
X_train_val = X_train_val/255

X_test = X_test/255

In [ ]:
X_train_val.describe()

Awesome. Done. Feel free to check that all grayscale values are properly scaled.

**Task 2**

Using all 784 features (no additional feature engineering or manipulation needed), start GridSearching for the best performing hyperparameters for 

KNN, DecisionTree, RandomForest, AdaBoost 

**Task 3**

When you have the best performing hyperparameters, do an ordinary train/validation split on X_train_val and y_train_val.

Train the models with their respective best performing hyperparameters on X_train and then evaluate performance on X_val.

In particular, make good use of confusion matrices here.

**Task 4**

For the models you've trained in Task 3, try plotting the validation samples they classified **incorrectly**. As the title for each image, write the true label and the predicted label.

Do the mistakes kinda make sense? Can you forgive your models? :)

**Task 5**

For each model with the identified best performing hyperparameters, do a full training on X_train_val - then assess performance on X_test.

Which model did you find to perform worst/best? Is the difference large?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

*We're going to pretend that the default hyperparameters are best here...*

Train and predict with KNN

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train_val, y_train_val)

In [ ]:
knn_pred = knn.predict(X_test)

In [ ]:
knn_pred

In [ ]:
knn_pred_probabilities = knn.predict_proba(X_test)

Train and predict with RF

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train_val, y_train_val)

In [ ]:
rf_pred = rf.predict(X_test)

In [ ]:
rf_pred_probabilities = rf.predict_proba(X_test)

Evaluate KNN

In [ ]:
y_test.value_counts()

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=knn_pred.reshape(-1,1))

cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

cm_display.plot()
plt.show() 

In [ ]:
accuracy = accuracy_score(y_test, knn_pred)
print("Accuracy:", accuracy, end='\n\n')

report = classification_report(y_test, knn_pred)
print("Classification Report:\n", report)

Evaluate RF

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=rf_pred.reshape(-1,1))

cm_display = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

cm_display.plot()
plt.show() 

In [ ]:
accuracy = accuracy_score(y_test, rf_pred)
print("Accuracy:", accuracy, end='\n\n')

report = classification_report(y_test, rf_pred)
print("Classification Report:\n", report)

Let's plot missclassifcations made by the RF, for some error analysis

In [ ]:
misclassified_indices = np.where(rf_pred != y_test.values.ravel())[0]

misclassified_indices

In [ ]:
np.random.shuffle(misclassified_indices)

num_images_to_display = 10

num_displayed = 0

for i in misclassified_indices:
    if num_displayed >= num_images_to_display:
        break

    predicted_class = rf_pred[i]
    correct_class = y_test.values.ravel()[i]

    predicted_class_probability = rf_pred_probabilities[i][predicted_class]
    correct_class_probability = rf_pred_probabilities[i][correct_class]
    
    plt.figure()
    plt.imshow(test_images[i], cmap='gray')
    plt.title(f"Predicted as {predicted_class}, Actual: {correct_class}\n\n"
              f"Probabilities per class\n" 
              f"Class {predicted_class} : {predicted_class_probability}, Class {correct_class} : {correct_class_probability}\n\n"
              'All probabilities\n'
              f"{rf_pred_probabilities[i]}")
    plt.axis('off')
    plt.show()
    
    num_displayed += 1

**Task 6**

If you feel frisky, you can try a tad more difficult problem. Try loading and training on the [Fashion MNIST](https://www.kaggle.com/datasets/zalando-research/fashionmnist) dataset instead. 

It's a relatively more difficult dataset to classify.